## Linking with Spark
pyspark는 driver 와 worker들의 python version 이 동일해야한다.

In [1]:
from pyspark import SparkContext, SparkConf

## Initializing Spark 
### SparkContext
어떻게 cluster 에 접근할지 Spark 에게 알려줌
### SparkConf
내 application 정보 저장  
### SparkSession 와 SparkConf 차이?
#### 공통점
Spark 앱 진입점  
<br>

#### 차이점
SparkSession 은 2.0 부터 도입된 새로운 진입점  
SparkContext 는 RDD API 중심 지원, SparkSession 은 RDD + DataFrame + SQL 등 모든 API 지원  
SparkSession 내부에 SparkContext 포함(spark.sparkContext)

In [2]:
conf = SparkConf().setAppName("RDD Programming Guide").setMaster("local")
sc = SparkContext(conf=conf)

25/04/18 00:21:38 WARN Utils: Your hostname, gimjunhaui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.45.202 instead (on interface en0)
25/04/18 00:21:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/18 00:21:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Using shell

shell 에서 사용법은 다음에   
python driver 설정 내용이 있는데 필요할 때 알아보자

## Resilient Distributed Datasets (RDDs)
RDDs 는 고장에 강하고 병렬로 처리할 수 있는 데이터 요소들의 집합  
RDDs 생성은 드라이버 프로그램에 있는 데이터 요소들 혹은 외부 저장장치에 있는 데이터셋 (HDFS, HBase, Hadoop 기반)

### Parallelized Collection
Parallelized Collection은 드라이버 프로그램에서 기존의 리스트나 반복 가능한 객체에 대해 SparkContext의 parallelize 메소드를 호출함으로 생성  
데이터 요소들은 복사되어 병렬로 처리할 수 있는 RDDs 가 됨

In [23]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
distData

MemoryError: 

일단 생성을 하면 분산된 데이터셋(distData) 는 병렬로 처리 가능  
예를 들어 아래와 같이 코드를 작성하면 모든 요소의 합을 구할 수 있다.

In [18]:
distData.reduce(lambda a, b: a + b)

15

Parallelized Collection 에서 가장 중요한 요소(변수) 하나는 데이터셋을 몇개의 파티션으로 나눌지이다.  
Spark 는 각 클러스터의 파티션에 대해 한가지 task 만 실행  
보통 클러스터의 CPU 당 2~4 파티션으로 나눈다. 
Spark 는 자동으로 클러스터 기반으로 파티션 갯수를 나눈다.  
그러나, 수동으로 몇개의 파티션을 나눌지 설정이 가능

In [19]:
sc.parallelize(data, 10) # 10개의 파티션으로 나눈다

ParallelCollectionRDD[30] at readRDDFromFile at PythonRDD.scala:289

In [2]:
# 실험 시간 갯수
# 1~1000억 하니 컴퓨터 메모리가 못버틴다...
# 메모리 에러 발생 -> SparkConf 기본 메모리가 1G 로 되어 있어서 아래와같이 설정
data = list(range(100000000))
conf = SparkConf().setAppName("MyApp").set("spark.executor.memory", "2g").set("spark.driver.memory", "2g")
sc = SparkContext(conf=conf)

25/04/18 00:22:21 WARN Utils: Your hostname, gimjunhaui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.45.202 instead (on interface en0)
25/04/18 00:22:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/18 00:22:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
distData1 = sc.parallelize(data, 2)
distData1.reduce(lambda a, b: a + b)

25/04/18 00:19:01 WARN TaskSetManager: Stage 0 contains a task of very large size (245355 KiB). The maximum recommended task size is 1000 KiB.


4999999950000000

In [3]:
distData2 = sc.parallelize(data, 10)
distData2.reduce(lambda a, b: a + b)

25/04/18 00:22:37 WARN TaskSetManager: Stage 0 contains a task of very large size (48972 KiB). The maximum recommended task size is 1000 KiB.


4999999950000000

#### 2개 클러스터 vs 10개 클러스터 시간 비교
1~1억 이 들어있는 List 의 sum 을 하나씩 더해서 구하는 경우  
2개 클러스터 24.832초  
10개 클러스터 17.91초  
나눠서 하면 빠르다!

### External Datasets
PySpark은 Hadoop 에서 호환되는 모든 저장 source로 분산된 데이터셋 생성이 가능 (local file system, HDFS, Cassandra, HBase, AWS S3 등)  
Spark 는 텍스트파일 sequence 파일 혹은 다른 Hadoop 입력형식을 지원한다.  

<strong>textFile</strong> method 을 사용해서 텍스트파일 RDDs 생성이 가능  
이 method 은 파일의 URI(로컬 경로, hdfs://, s3a:// 등의 URI) 를 가지고 줄 단위로 읽는다.  
생성된 RDDs 텍스트파일은 dataset 기능을 사용가능하다.

In [20]:
distFile = sc.textFile("../data/data.txt")
# distFile(data.txt) 의 총 길이
distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)

999